In [4]:
import os
import sys
import time
import numpy as np
import imgaug
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils
import zipfile
import urllib.request
import shutil

from form.dataset import CocoDataset
from form.utils import *

# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
DEFAULT_DATASET_YEAR = "2014"

In [9]:
class CocoConfig(Config):
    """Configuration for training on MS COCO.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = "Coco-SportsBall-Only"
    IMAGES_PER_GPU = 2
    NUM_CLASSES = 2
    STEPS_PER_EPOCH = 1000
#     TRAIN_BN = True
#     RPN_ANCHOR_RATIOS = [1]
#     DETECTION_MAX_INSTANCES = 10
#     DETECTION_MIN_CONFIDENCE = 0.85
    

In [10]:
from keras.callbacks import TensorBoard
from datetime import datetime

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = TensorBoard(log_dir=logdir,
                          histogram_freq=0,
                          write_graph=True,
                          write_images=False)

In [11]:
from keras.callbacks import ModelCheckpoint

filepath= "logs/checkpoints/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
config = CocoConfig()
config.display()

model = modellib.MaskRCNN(mode="training",
                          config=config,
                          model_dir=DEFAULT_LOGS_DIR)

model_path = COCO_MODEL_PATH

# Load weights
print("Loading weights ", model_path)
model.load_weights(model_path,
                   by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_bbox_loss': 1.0, 'rpn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [ ]:
# Training dataset. Use the training set and 35K from the
# validation set, as as in the Mask RCNN paper.
dataset_train = CocoDataset()
dataset_train.load_coco('./cocodataset', "train", year=DEFAULT_DATASET_YEAR, auto_download=True)
dataset_train.load_coco('./cocodataset', "valminusminival", year=DEFAULT_DATASET_YEAR, auto_download=True)
dataset_train.prepare()

In [ ]:
# Validation dataset
dataset_val = CocoDataset()
val_type = "val" if DEFAULT_DATASET_YEAR in '2017' else "minival"
dataset_val.load_coco('./cocodataset', val_type, year=DEFAULT_DATASET_YEAR, auto_download=True)
dataset_val.prepare()

In [ ]:
# Image Augmentation
# Right/Left flip 50% of the time
augmentation = imgaug.augmenters.Fliplr(0.5)

In [ ]:
# *** This training schedule is an example. Update to your needs ***

import tensorflow as tf
from keras.utils import CustomObjectScope

with CustomObjectScope({'tf': tf}):
    # Training - Stage 1
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=40,
                layers='heads',
                augmentation=augmentation,
                custom_callbacks=[tensorboard])

In [ ]:
from keras.utils import CustomObjectScope

with CustomObjectScope({'tf': tf}):
    # Training - Stage 2
    # Finetune layers from ResNet stage 4 and up
    print("Fine tune Resnet stage 4 and up")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=120,
                layers='4+',
                augmentation=augmentation,
                custom_callbacks=[tensorboard])

In [ ]:
from keras.utils import CustomObjectScope

with CustomObjectScope({'tf': tf}):
    # Training - Stage 3
    # Fine tune all layers
    print("Fine tune all layers")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE / 10,
                epochs=160,
                layers='all',
                augmentation=augmentation,
                custom_callbacks=[tensorboard, checkpoint])        

In [ ]:
from datetime import datetime
import os

model_path = os.path.join(DEFAULT_LOGS_DIR, "models/" + datetime.now().strftime("%Y%m%d-%H%M%S") + "mask_rcnn_ball.h5")
model.keras_model.save_weights(model_path)

In [ ]:
print("done")